In [1]:
import pandas as pd
import json
import os
import numpy as np
import time
from datetime import timedelta

import importlib

from pymongo import MongoClient

client = MongoClient('localhost', 27017) # change the ip and port to your mongo database's
db = client['kagglecrawler']
search_log_coll = db['search_log']
download_log_coll = db['download_log']
raw_notebook_coll = db['raw_notebooks']
task_log_coll = db['task_log']
central_task_log_coll = db['central_task_log']
notebook_metadata_coll = db['notebook_metadata']
old_notebook_metadata_coll = db['notebook_metadata_coll']
QUERY_FILE = os.path.join(os.getcwd(), 'Queries/pwc_queries.csv')


In [3]:
old_notebook_metadata_coll.name

'notebook_metadata_coll'

In [5]:
df_metadata = pd.DataFrame.from_dict(list(old_notebook_metadata_coll.find()))
df_metadata.drop(columns=['_id'], inplace=True)
df_metadata

,languageNullable,kernelTypeNullable,isPrivateNullable,enableGpuNullable,enableInternetNullable,id,ref,title,author,slug,...,hasIsPrivate,enableGpu,hasEnableGpu,enableInternet,hasEnableInternet,categoryIds,datasetDataSources,kernelDataSources,competitionDataSources,totalVotes
0,python,notebook,False,False,False,8196149,davidbnn92/task-tagging,Task Tagging,Davide Bonin,task-tagging,...,True,False,True,False,True,"[exploratory data analysis, feature engineerin...",[],[],[competitions/abstraction-and-reasoning-challe...,98
1,python,notebook,False,False,False,754846,samlac79/beer-recipe-exploratory-analysis,Beer Recipe Exploratory Analysis,Samuel Lachance,beer-recipe-exploratory-analysis,...,True,False,True,False,True,"[data visualization, beginner, exploratory dat...",[datasets/jtrofe/beer-recipes],[],[],56
2,python,notebook,False,False,True,23138244,prateekiet/shrinkage-limit-prediction,shrinkage_limit_prediction,Prateek Pal,shrinkage-limit-prediction,...,True,False,True,True,True,[],[datasets/prateekiet/soil-features],[],[],7
3,python,notebook,False,False,True,11959099,chayan8/chronic-kidney-disease-explored,Chronic Kidney Disease - Explored !,Chayan Kathuria,chronic-kidney-disease-explored,...,True,False,True,True,True,"[exploratory data analysis, seaborn, plotly]",[datasets/mansoordaku/ckdisease],[],[],46
4,python,notebook,False,False,False,6265299,malekzadeh/human-activity-recognition-with-mob...,human_activity_recognition_with_mobile_sensing,Mohammad Malekzadeh,human-activity-recognition-with-mobile-sensing,...,True,False,True,False,True,"[pandas, numpy, matplotlib]",[datasets/uciml/human-activity-recognition-wit...,[],[],17
5,rmarkdown,script,False,False,False,802154,ambarish/extensive-analysis-of-beer-recipes-wi...,Extensive Analysis of Beer Recipes with XGB Model,Bukun,extensive-analysis-of-beer-recipes-with-xgb-model,...,True,False,True,False,True,"[exploratory data analysis, xgboost]",[datasets/jtrofe/beer-recipes],[],[],25
6,rmarkdown,script,False,False,True,9736702,alexismayer/gravity-exploration-of-trade-betwe...,Gravity - Exploration of trade between countries,Alexis,gravity-exploration-of-trade-between-countries,...,True,False,True,True,True,[],[],[],[],5
7,python,notebook,False,False,True,11837854,aditi81k/chronic-kidney-disease-eda,Chronic Kidney Disease EDA,Aditi Kothiya,chronic-kidney-disease-eda,...,True,False,True,True,True,"[exploratory data analysis, classification, da...",[datasets/colearninglounge/chronic-kidney-dise...,[],[],22
8,python,notebook,False,False,False,391162,ezietsman/kepler-dataset-exploratory-analysis,Kepler Dataset Exploratory Analysis,EZ,kepler-dataset-exploratory-analysis,...,True,False,True,False,True,[],[datasets/nasa/kepler-exoplanet-search-results],[],[],16


In [10]:
records = df_metadata.to_dict('records')
len(records)

9

In [6]:
from kagglecrawler.distributed_notebook_crawling import KaggleNotebookCrawler

# Create crawler
kwargs = {
    'search_log_coll': search_log_coll, 
    'download_log_coll': download_log_coll, 
    'central_log_coll': download_log_coll, 
    'task_log_coll': task_log_coll, 
    'raw_notebook_coll': raw_notebook_coll, 
    'notebook_metadata_coll': notebook_metadata_coll
}
crawler = KaggleNotebookCrawler(**kwargs)